In [1]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
conn=ps.connect(dbname="MyDBs",
                user="postgres",
                password="****",
                host="localhost",
                port="5432"
)

In [50]:
sql="""select * from covid """
df=sqlio.read_sql_query(sql,conn)
df

,Province,Country/Region,Latitude,Longitude,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0
1,Afghanistan,Afghanistan,33.939110,67.709953,2020-01-23,0,0,0
2,Afghanistan,Afghanistan,33.939110,67.709953,2020-01-24,0,0,0
3,Afghanistan,Afghanistan,33.939110,67.709953,2020-01-25,0,0,0
4,Afghanistan,Afghanistan,33.939110,67.709953,2020-01-26,0,0,0
...,...,...,...,...,...,...,...,...
78381,Zimbabwe,Zimbabwe,-19.015438,29.154857,2021-06-09,111,5,161
78382,Zimbabwe,Zimbabwe,-19.015438,29.154857,2021-06-10,64,4,23
78383,Zimbabwe,Zimbabwe,-19.015438,29.154857,2021-06-11,192,3,30
78384,Zimbabwe,Zimbabwe,-19.015438,29.154857,2021-06-12,164,3,22


1) Write a code to check NULL values

In [38]:
#Checking each of the column if any of the colum has null values
sql="""select * from covid
where "Province" is null or
    "Country/Region" is null or
    "Latitude" is null or
    "Longitude" is null or
    "Date" is null or
    "Confirmed" is null or
    "Deaths" is null or
    "Recovered" is null;
"""
sqlio.read_sql_query(sql,conn)

,Province,Country/Region,Latitude,Longitude,Date,Confirmed,Deaths,Recovered


2) If NULL values are present, update them with zeros for all columns.

In [7]:
#We dont have any null values in the dataset

3) check total number of rows

In [8]:
sql="""
select count(*) from covid    
"""
sqlio.read_sql_query(sql,conn)

,count
0,78386


4) Check what is start_date and end_date

In [9]:
sql="""
select min("Date") as start_date ,max("Date") as end_date from covid    
"""
sqlio.read_sql_query(sql,conn)

,start_date,end_date
0,2020-01-22,2021-06-13


5) Number of month present in dataset

In [10]:
sql="""
select count(*) from 
(select Extract("month" from "Date") as months,count(*) as cnts from covid
group by 1)
"""
sqlio.read_sql_query(sql,conn)

,count
0,12


6) Find monthly average for confirmed, deaths, recovered

In [11]:
sql="""
SELECT date_part('Month',"Date") as month,
avg("Deaths") as average_deaths,
avg("Recovered") as average_recovered,
avg("Confirmed") as average_confirmed
FROM covid
group by 1
order by 1;
"""
sqlio.read_sql_query(sql,conn)

,month,average_deaths,average_recovered,average_confirmed
0,1.0,63.681185,1451.455496,2958.281438
1,2.0,34.277740,769.103440,1203.118706
2,3.0,33.930247,840.079912,1538.963762
3,4.0,59.980519,1623.213636,2602.577814
4,5.0,53.530582,2162.902074,2290.051948
5,6.0,40.835699,1220.153277,1357.885231
6,7.0,35.109552,983.058232,1432.361123
7,8.0,37.536657,1299.294721,1611.842899
8,9.0,34.777273,1438.906710,1784.587446
9,10.0,36.758274,1420.643067,2412.199623


7) Find most frequent value for confirmed, deaths, recovered each month

In [12]:
sql="""
with deaths as --> CTE from Deaths
(select "month","Deaths" from
(select date_part('month',"Date") as month,"Deaths",
ROW_NUMBER() over(partition by date_part('month',"Date") ORDER BY COUNT(*) DESC )
as rn
from covid
group by 1,2
)where rn=1)

,confirmed as --> CTE from Confirmed
(select "month","Confirmed" from
(select date_part('month',"Date") as month,"Confirmed",
ROW_NUMBER() over(partition by date_part('month',"Date") ORDER BY COUNT(*) DESC )
as rn
from covid
group by 1,2)
where rn=1)

,recovered as --> CTE from Deaths
(select "month","Recovered" from
(select date_part('month',"Date") as month,"Recovered",
ROW_NUMBER() over(partition by date_part('month',"Date") ORDER BY COUNT(*) DESC )
as rn
from covid
group by 1,2)
where rn=1	
)
select d.month,d."Deaths",r."Recovered",co."Confirmed" from deaths as d 
inner join recovered as r on r.month=d.month
inner join confirmed as co on r.month=co.month


"""
sqlio.read_sql_query(sql,conn)

,month,Deaths,Recovered,Confirmed
0,1.0,0,0,0
1,2.0,0,0,0
2,3.0,0,0,0
3,4.0,0,0,0
4,5.0,0,0,0
5,6.0,0,0,0
6,7.0,0,0,0
7,8.0,0,0,0
8,9.0,0,0,0
9,10.0,0,0,0


8) Find minimum values for confirmed, deaths, recovered per year

In [52]:
sql="""
SELECT date_part('Month',"Date") as month,
min("Deaths") as minimum_deaths,
min("Recovered") as minimum_recovered,
min("Confirmed") as minimum_confirmed
FROM covid
group by 1
order by 1;
"""
sqlio.read_sql_query(sql,conn)

,month,minimum_deaths,minimum_recovered,minimum_confirmed
0,1.0,0,0,0
1,2.0,0,0,0
2,3.0,0,0,0
3,4.0,0,0,0
4,5.0,0,0,0
5,6.0,0,0,0
6,7.0,0,0,0
7,8.0,0,0,0
8,9.0,0,0,0
9,10.0,0,0,0


9) Find maximum values of confirmed, deaths, recovered per year

In [53]:
sql="""
SELECT date_part('Month',"Date") as month,
max("Deaths") as maximum_deaths,
max("Recovered") as maximum_recovered,
max("Confirmed") as maximum_confirmed
FROM covid
group by 1
order by 1;
"""
sqlio.read_sql_query(sql,conn)

,month,maximum_deaths,maximum_recovered,maximum_confirmed
0,1.0,4475,87090,300462
1,2.0,3907,98389,134975
2,3.0,3869,102138,100158
3,4.0,4249,299988,401993
4,5.0,4529,422436,414188
5,6.0,7374,231456,134154
6,7.0,1595,140050,75866
7,8.0,1505,95881,85687
8,9.0,1703,101468,97894
9,10.0,3351,388340,99264


10) The total number of case of confirmed, deaths, recovered each month


In [54]:
sql="""
SELECT date_part('Month',"Date") as month,
sum("Deaths") as total_deaths,
sum("Recovered") as total_recovered,
sum("Confirmed") as total_confirmed
FROM covid
group by 1
order by 1;
"""
sqlio.read_sql_query(sql,conn)

,month,total_deaths,total_recovered,total_confirmed
0,1.0,402083,9164490,18678589
1,2.0,300890,6751190,10560976
2,3.0,323966,8021083,14694026
3,4.0,554220,14998494,24047819
4,5.0,511110,20651389,21865416
5,6.0,270414,8079855,8991916
6,7.0,167613,4693120,6838092
7,8.0,179200,6202833,7694938
8,9.0,160671,6647749,8244794
9,10.0,175484,6782150,11515841


11)  Check how corona virus spread out with respect to confirmed case(Eg.: total confirmed cases, their average, variance & STDEV )

In [66]:
sql="""
SELECT "Country/Region" as "Country/Region",
sum("Confirmed") as total_confirmed_cases,
avg("Confirmed") as average_confirmed_cases,
ROUND(stddev_pop("Confirmed"),2) as standard_deviation,
ROUND(variance("Confirmed"),2) as variance
FROM covid
group by 1
order by 1;
"""
sqlio.read_sql_query(sql,conn)

,Country/Region,total_confirmed_cases,average_confirmed_cases,standard_deviation,variance
0,Afghanistan,89861,176.544204,281.13,79192.20
1,Algeria,133388,262.058939,227.42,51822.18
2,Argentina,4124190,8102.534381,8365.13,70113173.96
3,Australia,30365,7.457024,40.26,1621.55
4,Austria,648620,1274.302554,1626.37,2650294.08
...,...,...,...,...,...
116,Vietnam,10630,20.884086,60.15,3624.56
117,West Bank and Gaza,311534,612.051081,688.00,474278.52
118,Yemen,6863,13.483301,24.68,610.33
119,Zambia,111746,219.540275,387.02,150078.10


12) Check how corona virus spread out with respect to death case per month

In [68]:
sql="""
SELECT date_part('Month',"Date") as month,
sum("Deaths") as total_death_cases,
avg("Deaths") as average_death_cases,
ROUND(stddev_pop("Deaths"),2) as standard_deviation,
ROUND(variance("Deaths"),2) as variance
FROM covid
group by 1
order by 1;
"""
sqlio.read_sql_query(sql,conn)

,month,total_death_cases,average_death_cases,standard_deviation,variance
0,1.0,402083,63.681185,281.07,79012.04
1,2.0,300890,34.277740,186.68,34852.62
2,3.0,323966,33.930247,172.57,29785.05
3,4.0,554220,59.980519,260.57,67905.92
4,5.0,511110,53.530582,277.07,76775.78
5,6.0,270414,40.835699,215.04,46250.19
6,7.0,167613,35.109552,145.40,21144.58
7,8.0,179200,37.536657,152.55,23277.87
8,9.0,160671,34.777273,141.78,20107.12
9,10.0,175484,36.758274,132.59,17583.75


13)  Check how corona virus spread out with respect to recovered case

In [69]:
sql="""
SELECT "Country/Region" as "Country/Region",
sum("Recovered") as total_recovered_cases,
avg("Recovered") as average_recovered_cases,
ROUND(stddev_pop("Recovered"),2) as standard_deviation,
ROUND(variance("Recovered"),2) as variance
FROM covid
group by 1
order by 1;
"""
sqlio.read_sql_query(sql,conn)

,Country/Region,total_recovered_cases,average_recovered_cases,standard_deviation,variance
0,Afghanistan,61438,120.703340,222.69,49690.60
1,Algeria,92966,182.644401,162.90,26588.56
2,Argentina,3721350,7311.100196,8350.89,69874590.21
3,Australia,26737,6.566061,57.42,3297.84
4,Austria,634624,1246.805501,1559.73,2437534.24
...,...,...,...,...,...
116,Vietnam,4050,7.956778,18.33,336.54
117,West Bank and Gaza,303826,596.907662,702.49,494461.78
118,Yemen,3743,7.353635,15.42,238.36
119,Zambia,97756,192.055010,381.62,145921.24


14) Find Country having highest number of the Confirmed case

In [75]:
sql="""
SELECT "Country/Region" as "Country/Region",
sum("Confirmed") as total_confirmed_cases
from covid
group by 1
order by total_confirmed_cases desc
limit(1)
"""
sqlio.read_sql_query(sql,conn)

,Country/Region,total_confirmed_cases
0,US,33461982


15) Find Country having lowest number of the death case

In [80]:
sql="""
SELECT "Country/Region" as "Country/Region",
sum("Deaths") as total_death_cases
from covid
group by 1
order by total_death_cases 
limit(1)
"""
sqlio.read_sql_query(sql,conn)

,Country/Region,total_death_cases
0,Kiribati,0


16) Find top 5 countries having highest recovered case

In [79]:
sql="""
SELECT "Country/Region" as "Country/Region",
sum("Recovered") as total_recovered_cases
from covid
group by 1
order by total_recovered_cases desc
limit(5)
"""
sqlio.read_sql_query(sql,conn)

,Country/Region,total_recovered_cases
0,India,28089649
1,Brazil,15400169
2,US,6303715
3,Turkey,5202251
4,Russia,4745756
